In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from creme import compat
from creme import datasets
from creme import metrics
from creme import model_selection
from creme import preprocessing
from torch import nn
from torch import optim

X_y = datasets.TrumpApproval()

n_features = 6
net = nn.Sequential(
    nn.Linear(n_features, 3),
    nn.Linear(3, 1),
)

model = (
    preprocessing.StandardScaler() |
    compat.PyTorch2CremeRegressor(
        net=net,
        loss_fn=nn.MSELoss(),
        optimizer=optim.SGD(net.parameters(), lr=.001),
        batch_size=1
    )
)
metric = metrics.MAE()

model_selection.progressive_val_score(X_y, model, metric)

MAE: 1.411203

In [54]:
from creme import base
from creme import metrics
from creme import preprocessing
from vowpalwabbit import pyvw


class VW2CremeBase(pyvw.vw):
    
    def _to_vw(self, x):
        return ' '.join(f'{k}:{v}' for k, v in x.items())
    
    def fit_one(self, x, y):
        #print(f'{y} | {self._to_vw(x)}')
        ex = self.example(f'{y} | {self._to_vw(x)}')
        self.learn(ex)
        self.finish_example(ex)
        return self
    
class VW2CremeRegressor(VW2CremeBase, base.Regressor):
    
    def predict_one(self, x):
        ex = self.example(f' | {self._to_vw(x)}')
        #ex = self.example(self._to_vw(x))
        y_pred = self.predict(ex)
        self.finish_example(ex)
        return y_pred
    
X_y = datasets.TrumpApproval()
    
metric = metrics.MAE()
        
model = preprocessing.StandardScaler() | VW2CremeRegressor(sgd=True)
        
for x, y in X_y:
    y_pred = model.predict_one(x)
    #print(y_pred)
    model.fit_one(x, y)
    metric.update(y, y_pred)
    
metric

MAE: 5.76122

In [4]:
class VW2CremeBinaryClassifier(VW2CremeBase, base.BinaryClassifier):
    
    def __init__(self, **params):
        params['binary'] = True
        params['link'] = 'logistic'
        params['loss_function'] = 'logistic'
        super().__init__(**params)
        
    def fit_one(self, x, y):
        y = int(y)
        if y == 0:
            y = -1
        return super().fit_one(x, y)
    
    def predict_one(self, x):
        ex = self.example(self._to_vw(x))
        y_pred = self.predict(ex)
        self.finish_example(ex)
        return y_pred
    

X_y = datasets.Elec2()
    
metric = metrics.LogLoss()
        
model = VW2CremeBinaryClassifier()
        
for x, y in X_y:
    y_pred = model.predict_one(x)
    model.fit_one(x, y)
    metric.update(y, y_pred)
    
metric

LogLoss: 19.875959

In [9]:
class VW2CremeMultiClassifier(VW2CremeBase, base.MultiClassifier):
    
    def __init__(self, **params):
        params['oaa'] = 2
        params['loss_function'] = 'logistic'
        params['probabilities'] = True
        super().__init__(**params)
        self.labels = {}
        
    def fit_one(self, x, y):
        try:
            y = self.labels[y]
        except KeyError:
            self.labels[y] = len(self.labels) + 1
            y = len(self.labels)
        return super().fit_one(x, y)
    
    def predict_one(self, x):
        ex = self.example('| ' + self._to_vw(x))
        y_pred = self.predict(ex)
        self.finish_example(ex)
        return y_pred
    

X_y = datasets.Elec2()
    
metric = metrics.LogLoss()
        
model = VW2CremeMultiClassifier()
        
for x, y in X_y:
    y = int(y) + 1
    y_pred = model.predict_one(x)
    model.fit_one(x, y)
    metric.update(y, y_pred)
    
metric

RuntimeError: label 0 is not allowed for multiclass.  Valid labels are {1,k}